In [9]:
import accelerate
import datasets
from datasets import(
load_dataset, 
load_dataset_builder,
get_dataset_split_names,
get_dataset_config_names)
import evaluate
import numpy as np
import peft
from peft import(
LoftQConfig,
LoraConfig,
get_peft_model)
import pickle
import os
import transformers
from transformers import(
AutoModelForCausalLM,
AutoTokenizer,
TrainingArguments)

In [14]:
current_dir=os.getcwd()
pkl_dir=os.path.join(current_dir, "pkl_files")
pkl_files=os.listdir(pkl_dir)

In [16]:
lla_model321=AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B", num_labels=3, torch_dtype="auto")
file_path=os.path.join(current_dir, "pkl_files", "llama3_2.pkl")
with open(file_path, "wb") as f:
    pickle.dump(lla_model321, f)